### poc and wip for multivar regression

# Imports and setup

General imports

In [ ]:
import pandas as pd

OpenGrid-specific imports

In [ ]:
from opengrid.library import houseprint
from opengrid import config
from opengrid.library import linearregression

c = config.Config()

Plotting settings

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

# Load Data

We are going to use gas consumption data and weather data. Because we don't want to overload the weather API, we will only use 1 location (Ukkel).

First, let's define the start and end date of our experiment. Let's take 1 year worth of data, starting with last month.

In [ ]:
end = pd.Timestamp.today().replace(day=1).normalize() + pd.Timedelta(days=1)
start = (end.replace(year=end.year-1)) - pd.Timedelta(days=2)

start = start.tz_localize('Europe/Brussels')
end = end.tz_localize('Europe/Brussels')
print(start, end)

## Gas Data

In [ ]:
# Load the Houseprint, and sync all data
hp = houseprint.Houseprint()
#hp = houseprint.load_houseprint_from_file('cache_hp.hp')
hp.init_tmpo()

In [ ]:
hp.sync_tmpos()

In [ ]:
#hp.save('cache_hp.hp')

In [ ]:
def gas_data_generator4():
    # Preferred method: as accurate as 3, and faster
    # Daily approach, obtain fully correct daily data.
    # To be aggregated to monthly or weekly or ...
    
    for gas_sensor in hp.get_sensors(sensortype='gas'):
        df = gas_sensor.get_data(head=start, tail=end, resample='day', unit='kWh', diff=False, tz='Europe/Brussels')
        df = df.diff().shift(-1).dropna()
        if df.empty:
            continue
        yield df

Let's have a peek

In [ ]:
gas_data = gas_data_generator4()

In [ ]:
peek = next(gas_data)
peek.plot()

## Weather Data

Run this block to download the weather data and save it to a pickle. This is a large request, and you can only do 2 or 3 of these per day before your credit with Forecast.io runs out!

To get the data run the cell below

In [ ]:
from opengrid.library import forecastwrapper
weather = forecastwrapper.Weather(location='Ukkel, Belgium', start=start, end=end - pd.Timedelta(days=1))
weather_data = weather.days(heating_base_temperatures=[0, 6, 8 ,10, 12, 14, 16, 18]).dropna(axis=1)
weather_data.drop(['icon', 'summary'], axis=1, inplace=True)
weather_data = weather_data.applymap(float)

# Put data together

I wrote a generator that uses our previously defined generator so you can generate while you generate.

In [ ]:
def analysis_data_generator():
    gas_data = gas_data_generator4()
    for gas_df in gas_data:
        gas_df.name='gas'
        df = pd.concat([gas_df, weather_data], axis=1).dropna()
        df = df.tz_convert('Europe/Brussels')
        yield df

Let's have another peek

In [ ]:
analysis_data = analysis_data_generator()

In [ ]:
peek = next(analysis_data)
peek = peek.resample(rule='MS').sum()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot_date(peek.index, peek['gas'], '-', color='grey', lw=8, label='gas')
for column in peek.columns[1:]:
    if 'heatingDegreeDays' in column:
        ax2.plot_date(peek.index, peek[column], '-', label=column)
plt.legend()

# Run Regression Analysis



In [ ]:
%%bash
pip install statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as fm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
def find_best_rsquared(list_of_fits):
    """Return the best fit, based on rsquared"""
    res = sorted(list_of_fits, key=lambda x:x.rsquared)
    return res[-1]

def find_best_akaike(list_of_fits):
    """Return the best fit, based on Akaike information criterion"""
    res = sorted(list_of_fits, key=lambda x:x.aic)
    return res[0]

def find_best_bic(list_of_fits):
    """Return the best fit, based on Akaike information criterion"""
    res = sorted(list_of_fits, key=lambda x:x.bic)
    return res[0]
    


In [ ]:
from opengrid.library.analysis import Analysis
from copy import deepcopy
class multivar_regression(Analysis):
    
    def do_analysis(self, y, list_of_exog=None):
        if list_of_exog is None:
            list_of_exog = self.df.columns.tolist()
            list_of_exog.remove(y)
            
        
        list_of_fits = []
        # first fit is just the mean
        list_of_fits.append(fm.ols(formula='{} ~ 1'.format(y), data=self.df).fit())
        # try to improve the model until no improvements can be found
        all_exog=list_of_exog[:]
        while all_exog:
            # try each x in all_exog
            best_fit = deepcopy(list_of_fits[-1])
            for x in all_exog:
                # make new_fit, compare with best found so far
                formula = list_of_fits[-1].model.formula + '+{}'.format(x)
                fit = fm.ols(formula=formula, data=self.df).fit()
                best_fit = find_best_bic([best_fit, fit])
            
            # Sometimes, the obtained fit may be better, but contains unsignificant parameters.
            # Correct the fit by removing the unsignificant parameters and estimate again
            for par in best_fit.pvalues.where(best_fit.pvalues > 0.05).dropna().index:
                corrected_formula = best_fit.model.formula.replace('+{}'.format(par), '')
                best_fit = fm.ols(formula=corrected_formula, data=self.df).fit()
            
            # if best_fit is same as last fit in list_of_fits, exit
            if best_fit.model.formula == list_of_fits[-1].model.formula:
                break
            else:
                list_of_fits.append(best_fit)
                all_exog.remove(x)
                #print(u'{} ==> AIC={}, BIC={}, R²={}'.format(best_fit.model.formula, best_fit.aic, best_fit.bic, best_fit.rsquared))
        self.list_of_fits = list_of_fits
        # Add model results to data as column 'model'
        self.df['model'] = list_of_fits[-1].predict(self.df)
        
    def plot(self, show_summary=True):
        fit = self.list_of_fits[-1]
        if show_summary:
            print(fit.summary())
        # The first variable in the formula is the most significant.  Use it as abcis for the plot
        try:
            x = fit.model.formula.split('+')[1].strip()
        except IndexError:
            x = 'heatingDegreeDays16'
        
        plt.figure()
        plt.plot(self.df[x], self.df[fit.model.endog_names], 'ro', ms=8)
        # get sorted model values
        dfmodel = self.df[[x, 'model']]
        dfmodel.index = dfmodel[x]
        dfmodel.sort(inplace=True)
        plt.plot(dfmodel.index, dfmodel['model'], 'b--')
        plt.title('{} - rsquared={} - BIC={}'.format(fit.model.formula, fit.rsquared, fit.bic))
        plt.show()

                
            
    
                
            
            
            
            

In [ ]:
analysis_data = analysis_data_generator()
mrs = []
for data in analysis_data:  
    data = data.resample(rule='W').sum()
    mrs.append(multivar_regression(data, 'gas'))

    

In [ ]:
for mr in mrs:
    mr.plot()